In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time
from pandas.plotting import table 
import matplotlib.pyplot as plt
import string

# Borrowed the offset idea from this (much more professionally done) tableau page
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet
# No adjustments for similar names etc (apologies)

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html']+['https://www.janestreet.com/puzzles/archive/page'+str(i)+'/index.html' for i in range(2,13)]
links =  ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [20]:
outputs = []
for newcomer_offset in [6]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df['Name']=df['Name'].str.lstrip("1234567890. ")
    df['Name'] = df['Name'].str.title()
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
   
df2.sort_values(['Score','Solved'],ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Evan Semet,35,35,0.853659
Karl Mahlburg,53,59,0.815385
Senthil Rajasekaran,58,70,0.763158
Gareth Owen,41,50,0.732143
Aaditya Raghavan,43,53,0.728814
Lazar Ilic,38,47,0.716981
Blaine Hill,15,15,0.714286
Calvin Pozderac,62,83,0.696629
Danica Xiong,16,17,0.695652


In [22]:
df2.sort_values('Solved',ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Calvin Pozderac,62,83,0.696629
Senthil Rajasekaran,58,70,0.763158
Karl Mahlburg,53,59,0.815385
Sean Egan,52,77,0.626506
Keith Schneider,47,74,0.587500
Aaditya Raghavan,43,53,0.728814
Gareth Owen,41,50,0.732143
Heidi Stockton,40,96,0.392157
Lazar Ilic,38,47,0.716981


In [6]:
import xlwings as xw
sht = xw.Book().sheets[0]
sht.range('a1').options(index=False).value =  df

In [7]:
links

['https://www.janestreet.com/puzzles/current-puzzle/',
 'https://www.janestreet.com//puzzles/tree-edge-triage-solution',
 'https://www.janestreet.com//puzzles/many-happy-returns-solution',
 'https://www.janestreet.com//puzzles/altered-states-2-solution',
 'https://www.janestreet.com//puzzles/number-cross-4-solution',
 'https://www.janestreet.com//puzzles/robot-capture-the-flag-solution',
 'https://www.janestreet.com//puzzles/hooks-10-solution',
 'https://www.janestreet.com//puzzles/some-off-square-solution',
 'https://www.janestreet.com//puzzles/some-f-squares-solution',
 'https://www.janestreet.com//puzzles/hall-of-mirrors-2-solution',
 'https://www.janestreet.com//puzzles/knight-moves-5-solution',
 'https://www.janestreet.com//puzzles/a-weird-tour-solution',
 'https://www.janestreet.com//puzzles/getting-from-a-to-b-solution',
 'https://www.janestreet.com//puzzles/single-cross-2-solution',
 'https://www.janestreet.com//puzzles/choco-banana-solution',
 'https://www.janestreet.com//puzz